In [ ]:
# This code has generated by ChatGPT for the most part
# It has then been reviewed, corrected, commented and augmented by
# a humabn being (Eric).

import random
import heapq

# This model represents a memory hierarchy with
# - 3 levels of cache (L1, L2, L3)
# - a DDR memory implemening some simple optimization features (see the DDR class)
# - an interconnect
# The number of cores, levels of cache, characteristics of the cache (number of ways,...)
# are parameters and can be modified.



# -----------------------------------------------------
# CacheLine: Represents a single cache line in the cache hierarchy
# -----------------------------------------------------
# We are using n-way associative cache:
# - Each set contains n cache lines with a tag entry.
# - Each line contains m bytes.
#
# A memory Address is structured as follows:
# +----------------+-----------+-----------+
# |     Tag        |   Index   |  Offset   |
# +----------------+-----------+-----------+
# where
#
# (Used to find blk) (Set selection) (Byte in block)
# "Block" and "line" are used interchangeably.
# Cache Structure:
# Set 0: [Block 0] [Block 1] [Block 2] [Block 3]  ← 4-way associativity (n=4)
# Set 1: [Block 0] [Block 1] [Block 2] [Block 3]
# ...
# Set N: [Block 0] [Block 1] [Block 2] [Block 3]

class CacheLine:
    def __init__(self):
        self.valid = False       # Indicates if this line holds valid data
        self.tag = None          # Tag of the data block
        self.data = 0            # Simulated data content
        self.dirty = False       # Indicates if the line has been written to (for write-back)

# Implements Pseudo-LRU (PLRU) replacement policy for N-way set associative caches
# The pseudoi-LRU is used to determine the bock to replace in case of cache miss.
# A binary tree is used to implement the PLRU algorithm. There is one tree per set.
# For a 4-way cache, 3 bits are used to determine the block to select.
#
#      Bit 0 (Root)
#     /           \
#   Bit 1         Bit 2
#   /   \         /   \
# Block0 Block1 Block2 Block3
#
# Each node of the tree contains a direction (left=0, right=1) that indicates
# the path to follow to find the next pLRU entry.
class PLRU:
    def __init__(self, ways):
        self.bits = [0] * (ways - 1)  # Tree structure to track usage
        self.ways = ways

    # Update the binary tree in case of a hit
    # The bits in the tree are modified to point "away" from this entry
    # (which is the MRU)
    def update_on_access(self, way):
        idx = 0
        num_levels = self.ways.bit_length() - 1
        for level in range(num_levels):
            # Select direction according to the way
             # (e.g., way=3=0b101 in a 4-way cache => direction = 1 (right subtree), 0 (left subtree)
            direction = (way >> (num_levels - 1 - level)) & 1
            self.bits[idx] = 1-direction # Point to the opposite direction
            idx = (idx << 1)+ 1 + direction

    # Compute the next victim (the pLRU)
    # The block is selected by traversing the tree according
    # to the directions given by each bit.
    def get_victim(self):
        idx = 0
        way = 0
        for level in range(self.ways.bit_length() - 1):
            direction = self.bits[idx]
            way = (way << 1) | direction
            idx = ( idx << 1) + 1 + direction
        return way

# ---------------------------------------------------------
# Represents a memory access request (either read or write)
# ---------------------------------------------------------
class DDRRequest:
    def __init__(self, core_id, time, req_type, addr, callback=None, value=None):
        self.core_id = core_id
        self.time = time              # Time of request
        self.req_type = req_type      # 'read' or 'write'
        self.addr = addr
        self.callback = callback      # Callback function to return read result
        self.value = value            # Value to write (for write requests)

    def __lt__(self, other):
        return self.time < other.time

# ---------------------------------------------------------
# Interconnect model between CPU cores and DDR, with bandwidth and latency
# ---------------------------------------------------------
# Behaviour :
# - Each request takes at least some base delay to be served.
# - A request may be delayed if the interconnect bandwidh has been "used"
# The interconnect canot serve more than "bandwidth" request in one cycle.
# Note
# - Using a heapqueue ensures that all items are and remain sorted
#   according to their ready_time (and req)
#


class Interconnect:
    def __init__(self, memory, delay=5, bandwidth=4):
        self.memory = memory
        self.queue = []               # Queue of pending memory requests
        self.delay = delay            # Base delay before forwarding to DDR
        self.bandwidth = bandwidth    # Max number of requests per cycle
        self.cycle = 0

    # Push a request in the request queue.
    # We push the couple (ready_time, request) where ready_time is the earliest
    # time at which the request may be served.
    def request(self, req):
        print(f"[Cycle {self.cycle}] ➜ New CPU request queued: Core {req.core_id}, {req.req_type.upper()} Addr {req.addr}")
        ready_time = self.cycle + self.delay + random.randint(0, 2)
        heapq.heappush(self.queue, (ready_time, req))

    # Process the interconnect current cycle
    def tick(self):
        processed = 0
        temp_queue = []

        # Forward requests to DDR respecting bandwidth limit
        while self.queue and self.queue[0][0] <= self.cycle and processed < self.bandwidth:
            _, req = heapq.heappop(self.queue)
            self.memory.request(req)
            processed += 1

        #[TODO]: the following code seems useless since the queue is always
        #sorted...

        # Defer excess requests for next cycles
        while self.queue and self.queue[0][0] <= self.cycle:
            temp_queue.append(heapq.heappop(self.queue))
        for item in temp_queue:
            heapq.heappush(self.queue, item)

        self.cycle += 1


# ---------------------------------------------------------
#  DDR memory model with banks, row buffers, and latency variations
# Access latencies depend on the active bank
# Each bank has a row buffer. An access to the same row is faster. In case of
# miss, an access precharge and activate delay is added.
#
# The parameters are
# tRCD (Row to Column Delay)
# tRP (Row Precharge) : time to close one row of memory cell before opening another row
#                       in the same bank.
# tCAS (Column Access Strobe latency)
# tRC (Row Cycle time)
# Behaviour:
# - The DDR requests coming from the two cores are queued and arbitrated per cycle by the DDR controler.
# - The requests are re-ordered so that the best next-command is served first:
#   - prefer row hits
#   - avoid accesses to busy banks
# - The bank is determined by the address' LSB: row = addr % num_banks
# - Each row contains 16 addresses: row = addr // 16
# ---------------------------------------------------------
class DDRMemory:
    def __init__(self, latency=50, row_hit_latency=20, row_miss_penalty=30, precharge_time=10, num_banks=4):
        self.base_latency = latency
        self.row_hit_latency = row_hit_latency
        self.row_miss_penalty = row_miss_penalty
        self.precharge_time = precharge_time
        self.num_banks = num_banks
        self.bank_row_buffers = [None for _ in range(num_banks)]
        self.bank_precharge_end = [0 for _ in range(num_banks)]
        self.memory = {}
        self.cycle = 0

        self.pending = []  # Requests waiting to be scheduled
        self.scheduled = []  # Requests that have been scheduled for completion
        self.last_address_time = {}  # Track the latest cycle each address is scheduled to enforce order

    def _get_bank(self, addr):
        return addr % self.num_banks

    def _get_row(self, addr):
        return addr // 16  # Example: each row covers 16 addresses

    # Queue a request
    def request(self, req):
        print(f"[Cycle {self.cycle}] ➜ New DDR request queued: Core {req.core_id}, {req.req_type.upper()} Addr {req.addr}")
        self.pending.append((self.cycle, req))

    # Process the DDR current cycle
    def tick(self):
        self._issue_best_request()

        while self.scheduled and self.scheduled[0][0] <= self.cycle:
            _, req = heapq.heappop(self.scheduled)
            if req.req_type == 'read':
                val = self.memory.get(req.addr, 0)
                print(f"[Cycle {self.cycle}] ✔ READ COMPLETE (Core {req.core_id}, Addr {req.addr}) => {val}")
                if req.callback:
                    req.callback(val)
            elif req.req_type == 'write':
                self.memory[req.addr] = req.value
                print(f"[Cycle {self.cycle}] ✔ WRITE COMPLETE (Core {req.core_id}, Addr {req.addr}) <= {req.value}")

        self.cycle += 1

    # Return the "best" request from the request queue
    def _issue_best_request(self):

        # Is the pending request queue empty?
        if not self.pending:
            print(f"[Cycle {self.cycle}] No pending DDR requests to schedule.")
            return

        # Sort the pending requests according to their "score"
        self.pending.sort(key=lambda pair: self._score(pair[1], pair[0]))

        for i, (arrival_time, req) in enumerate(self.pending):
            bank = self._get_bank(req.addr)
            row = self._get_row(req.addr)
            last_time = self.last_address_time.get(req.addr, -1)

            if last_time >= self.cycle:
                continue

            if self.bank_precharge_end[bank] > self.cycle:
                continue  # Bank is busy

            if self.bank_row_buffers[bank] == row:
                delay = self.row_hit_latency
                row_status = "ROW HIT"
            else:
                delay = self.precharge_time + self.row_miss_penalty + self.row_hit_latency
                row_status = "ROW MISS"
                self.bank_row_buffers[bank] = row
                self.bank_precharge_end[bank] = self.cycle + self.precharge_time

            completion_time = self.cycle + delay
            req.completion_time = completion_time
            self.last_address_time[req.addr] = completion_time
            heapq.heappush(self.scheduled, (completion_time, req))
            self.pending.pop(i)

            print(f"[Cycle {self.cycle}] ➜ Scheduling {req.req_type.upper()} for Addr {req.addr} (Core {req.core_id})")
            print(f"                  ↳ Bank {bank}, Row {row} | {row_status} | Will complete at Cycle {completion_time}")
            break

    # Compute a score for each request
    def _score(self, req, arrival_time):
        bank = self._get_bank(req.addr)
        row = self._get_row(req.addr)
        score = arrival_time

        if self.bank_row_buffers[bank] == row:
            score -= 100  # Favor row hits
        elif self.bank_precharge_end[bank] > self.cycle:
            score += 50  # Penalize busy banks

        return score

#---------------------------------------
# Models one level in the cache hierarchy
#----------------------------------------
class CacheLevel:
    def __init__(self, level_name, core_id, size, line_size, assoc, memory=None, write_back=True, write_allocate=True):
        self.level = level_name
        self.core_id = core_id
        self.line_size = line_size
        self.assoc = assoc
        self.num_sets = (size // line_size) // assoc
        self.sets = [[CacheLine() for _ in range(assoc)] for _ in range(self.num_sets)]
        self.plru_trees = [PLRU(assoc) for _ in range(self.num_sets)]
        self.memory = memory        # Could be DDR or next cache level
        self.lower = None           # Lower level cache
        self.write_back = write_back
        self.write_allocate = write_allocate
        self.hits = 0
        self.misses = 0

    # Extract the set index from the address
    #  addr = [ tag ][ idx ][ offset ]
    def _index(self, addr):
        return (addr // self.line_size) % self.num_sets

    # Extract the tag from the address
    #  addr = [ tag ][ idx ][ offset ]
    def _tag(self, addr):
        return addr // (self.line_size * self.num_sets)

    # Handles cache read request
    def read(self, addr, callback):
        index = self._index(addr)
        tag = self._tag(addr)
        cache_set = self.sets[index]
        plru = self.plru_trees[index]

        # Seach the tag in the cache set
        for i, line in enumerate(cache_set):
            if line.valid and line.tag == tag:
                # There is a hit.
                print(f"[{self.level} cache hit for {addr} on core {self.core_id}]")
                self.hits += 1
                # Update the pLRU tree to point away from the MRU
                plru.update_on_access(i)
                callback(line.data)
                return

        print(f"[{self.level} cache miss for {addr} on core {self.core_id}]")
        # On miss, choose victim line using PLRU and fetch from lower memory
        self.misses += 1
        victim_idx = plru.get_victim()
        victim_line = cache_set[victim_idx]

        # If the victime line is valid and dirty, we have to write the data to
        # the next level of memory before loading the cache entry with the
        # data.
        def lower_cb(val):
            # Write evicted data if dirty
            if victim_line.valid and victim_line.dirty and self.write_back:
                victim_addr = ((victim_line.tag * self.num_sets) + index) * self.line_size
                if self.lower:
                    self.lower.write(victim_addr, victim_line.data)
                elif self.memory:
                    self.memory.request(DDRRequest(self.core_id, self.memory.cycle, 'write', victim_addr, value=victim_line.data))
            # Now that we have written the data to the next memory level, the
            # cache entry is updated with the new data (val)
            victim_line.valid = True
            victim_line.tag = tag
            victim_line.data = val
            victim_line.dirty = False
            plru.update_on_access(victim_idx)
            callback(val)

        # Forward the read request to the ower-level cache (if any)
        if self.lower:
            self.lower.read(addr, lower_cb)
        # Or to DDR...
        elif self.memory:
            self.memory.request(DDRRequest(self.core_id, self.memory.cycle, 'read', addr, lower_cb))

    # Handles cache write request
    def write(self, addr, val):
        index = self._index(addr)
        tag = self._tag(addr)
        cache_set = self.sets[index]
        plru = self.plru_trees[index]
        print(f"[{self.level} cache write for {addr} on core {self.core_id}]")

        for i, line in enumerate(cache_set):
            if line.valid and line.tag == tag:
                # There is a cache hit
                line.data = val
                # If the cache is write-back, the data will be written to
                # memory when evicted, so it is marked "dirty"
                line.dirty = True if self.write_back else False
                plru.update_on_access(i)
                # If the cache is write-through, the write operation is
                # propagated to the lower levels of the memory hierarchy
                if not self.write_back:
                    if self.lower:
                        self.lower.write(addr, val)
                    elif self.memory:
                        self.memory.request(DDRRequest(self.core_id, self.memory.cycle, 'write', addr, value=val))
                return

        # There is a cache miss...
        if self.write_allocate:
            # Find the entry to be evicted.
            victim_idx = plru.get_victim()
            victim_line = cache_set[victim_idx]
            # If we are in write-back mode and the cache line is dirty,
            # it has to be written to the lower level of the memory hierarchy
            # before being overwritten.
            if victim_line.valid and victim_line.dirty and self.write_back:
                victim_addr = ((victim_line.tag * self.num_sets) + index) * self.line_size
                if self.lower:
                    self.lower.write(victim_addr, victim_line.data)
                elif self.memory:
                    self.memory.request(DDRRequest(self.core_id, self.memory.cycle, 'write', victim_addr, value=victim_line.data))

            # The entry is now valid
            victim_line.valid = True
            victim_line.tag = tag
            victim_line.data = val
            # It is dirty (only necessary if write back is active)
            victim_line.dirty = self.write_back
            plru.update_on_access(victim_idx)
        else:
            # If write allocate is false, the data is written to the next level
            # of the memory hierachy.
            if self.lower:
                self.lower.write(addr, val)
            elif self.memory:
                self.memory.request(DDRRequest(self.core_id, self.memory.cycle, 'write', addr, value=val))

    def stats(self):
        total = self.hits + self.misses
        return {
            "level": self.level,
            "hits": self.hits,
            "misses": self.misses,
            "hit_rate": self.hits / total if total else 0
        }

# Models full multi-level cache hierarchy for a core
class MultiLevelCache:
    def __init__(self, core_id, l1_conf, l2_conf, l3_conf, interconnect):
        self.core_id = core_id
        self.interconnect = interconnect
        self.l1 = CacheLevel("L1", core_id, memory=None, **l1_conf)
        self.l2 = CacheLevel("L2", core_id, memory=None, **l2_conf)
        # Cache L3 is connected to the DDR memory
        self.l3 = CacheLevel("L3", core_id, memory=interconnect, **l3_conf)
        # Cache L1 is connected to cache L2
        self.l1.lower = self.l2
        # Cache L2 is connected to cache L3
        self.l2.lower = self.l3
        # Cache L3 has no lower level cache.
        self.l3.lower = None

    def read(self, addr, callback):
        self.l1.read(addr, callback)

    def write(self, addr, val):
        self.l1.write(addr, val)

    def stats(self):
        return {
            "core": self.core_id,
            "L1": self.l1.stats(),
            "L2": self.l2.stats(),
            "L3": self.l3.stats()
        }

# Simulation setup

random.seed(0)

ddr = DDRMemory()
interconnect = Interconnect(ddr, delay=5, bandwidth=4)
l1_conf = {'size': 32, 'line_size': 4, 'assoc': 2}
l2_conf = {'size': 128, 'line_size': 4, 'assoc': 4}
l3_conf = {'size': 512, 'line_size': 4, 'assoc': 8}

core0 = MultiLevelCache(0, l1_conf, l2_conf, l3_conf, interconnect)
core1 = MultiLevelCache(1, l1_conf, l2_conf, l3_conf, interconnect)

# Run a small simulation loop with random accesses
for cycle in range(100):
    print(f"\n========== CYCLE {cycle} ==========")
    addr = random.randint(0, 20)
    core = core0 if random.random() < 0.5 else core1
    if random.random() < 0.5:
        print(f"write at {addr} for core {core.core_id}")
        core.write(addr, random.randint(0, 1000))
    else:
        print(f"read at {addr} for core {core.core_id}")
        core.read(addr, lambda val: None)
    interconnect.tick()
    ddr.tick()

# Report results
print(core0.stats())
print(core1.stats())



========== CYCLE 0 ==========
write at 12 for core 1
[L1 cache write for 12 on core 1]
[Cycle 0] No pending DDR requests to schedule.

========== CYCLE 1 ==========
read at 16 for core 0
[L1 cache miss for 16 on core 0]
[L2 cache miss for 16 on core 0]
[L3 cache miss for 16 on core 0]
[Cycle 1] ➜ New CPU request queued: Core 0, READ Addr 16
[Cycle 1] No pending DDR requests to schedule.

========== CYCLE 2 ==========
read at 15 for core 0
[L1 cache miss for 15 on core 0]
[L2 cache miss for 15 on core 0]
[L3 cache miss for 15 on core 0]
[Cycle 2] ➜ New CPU request queued: Core 0, READ Addr 15
[Cycle 2] No pending DDR requests to schedule.

========== CYCLE 3 ==========
write at 16 for core 0
[L1 cache write for 16 on core 0]
[Cycle 3] No pending DDR requests to schedule.

========== CYCLE 4 ==========
read at 19 for core 1
[L1 cache miss for 19 on core 1]
[L2 cache miss for 19 on core 1]
[L3 cache miss for 19 on core 1]
[Cycle 4] ➜ New CPU request queued: Core 1, READ Addr 19
[Cycle 4]